**Programa Para o Twitter***

In [1]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkTwitterV02") \
    .getOrCreate()

23/01/31 22:51:25 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/31 22:51:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 22:51:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/31 22:51:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/31 22:51:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/31 22:51:26 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
#bibliotecas utilizadas para realizar a análise dos textos
from textblob import TextBlob  #utilizada para realizar o processamento do texto e análise de sentimento
from googletrans import Translator #utilizado para traduzir textos 
from unidecode import unidecode  #utilizada para "decodificar caracteres" não textuais

In [3]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
twitters = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9995) \
    .load()

23/01/31 22:51:28 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, FloatType

In [5]:
#função para realizar a tradução para o inglês 
def translate_udf(col):
    trans_obj=Translator().translate(col)
    return trans_obj.text

#função para realizar a análise de sentimento
def sentiment_udf(col):
    sentiment_text=TextBlob(col)
    return sentiment_text.polarity


In [6]:
#definição das função como User-Defined-Function
unicode_udf_string = udf(lambda z: unidecode(z), StringType()) #define a função de decode para ser utilizada no dataframe
group_by_sentiment = udf(lambda x: 'negativo' if x < -0.1 else 'positivo' if x > 0.1 else 'neutro',StringType())
translate_udf_string = udf(translate_udf, StringType()) #define a função de tradução
sentiment_udf_float = udf(sentiment_udf, FloatType()) #define a função de tradução

In [7]:
#testando funções
teste="Eu ♥ o meu cachorro, ele é o meu melhor amigo"
decode=unidecode(teste)
print(decode)
decodeEN=Translator().translate(decode)
print(decodeEN.text)
a=str(decodeEN)
sentiment = TextBlob(a)
print(sentiment.polarity)

Eu  o meu cachorro, ele e o meu melhor amigo
Me my dog, he is my best friend
1.0


In [8]:
#Aplica as funções udf para a seleção de colunas
twitters_unicode=twitters.select( "value",unicode_udf_string(twitters.value).alias("unicoded")) #decodifica
twitters_uni_trans=twitters_unicode.select( "value","unicoded",translate_udf_string(col("unicoded")).alias("twitter_EN")) #traduz
twitters_uni_trans_sent=twitters_uni_trans.select("value", "unicoded", "twitter_EN", sentiment_udf_float(col("twitter_EN")).alias("analise"))#análise de sentimento
t_sent_label= twitters_uni_trans_sent.select("value","unicoded","twitter_EN", "analise",group_by_sentiment(col("analise")).alias("classificacao"))

In [9]:
t_sent_count = t_sent_label.groupBy("classificacao").count() 

In [10]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
#Dando ERRO!! =(
query = t_sent_count \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination() #aguarda até que a "streaming query" termine 

23/01/31 22:51:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/hn/zqmlf1cd4s3_llkx270x0kw80000gn/T/temporary-723cd89f-b73a-45ff-bef2-bf08b31034ad. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/31 22:51:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


objc[92248]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called.
objc[92248]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called. We cannot safely call it or ignore it in the fork() child process. Crashing instead. Set a breakpoint on objc_initializeAfterForkError to debug.
objc[92249]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called.
objc[92249]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called. We cannot safely call it or ignore it in the fork() child process. Crashing instead. Set a breakpoint on objc_initializeAfterForkError to debug.
objc[92251]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called.
objc[92251]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called. We cannot safe

23/01/31 22:51:32 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:581)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:91)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon

23/01/31 22:51:32 WARN TaskSetManager: Lost task 4.0 in stage 0.0 (TID 4) (192.168.68.104 executor driver): TaskKilled (Stage cancelled)
23/01/31 22:51:32 WARN TaskSetManager: Lost task 1.0 in stage 0.0 (TID 1) (192.168.68.104 executor driver): TaskKilled (Stage cancelled)
23/01/31 22:51:32 WARN TaskSetManager: Lost task 5.0 in stage 0.0 (TID 5) (192.168.68.104 executor driver): TaskKilled (Stage cancelled)
23/01/31 22:51:32 WARN TaskSetManager: Lost task 3.0 in stage 0.0 (TID 3) (192.168.68.104 executor driver): TaskKilled (Stage cancelled)


StreamingQueryException: Query [id = 3bed2ecb-7c1c-49ec-9525-585eb1e6a28c, runId = 11834904-e87a-4cb7-90ef-254088e884ca] terminated with exception: Writing job aborted

23/01/31 22:51:32 WARN TaskSetManager: Lost task 2.0 in stage 0.0 (TID 2) (192.168.68.104 executor driver): TaskKilled (Stage cancelled)
